In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import spectral
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Input, Flatten, Reshape
from keras.optimizers import Adam
from keras.activations import relu, softmax
from keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [4]:
input_image = loadmat('./data2/Salinas_corrected.mat')['salinas_corrected']

In [7]:
input_image.shape

(512, 217, 204)

In [11]:
label = loadmat('./data2/Salinas_gt.mat')['salinas_gt']

In [12]:
label.shape

(512, 217)

In [32]:
label = label.reshape((d1*d2, 1))

In [33]:
def get_label_count(label_array):
    res = {}
    m, n = label_array.shape
    for i in range(m):
        for j in range(n):
            if label_array[i][j] not in res:
                res[label_array[i][j]] = 1
            else:
                res[label_array[i][j]] += 1
    return res

In [34]:
label_count = get_label_count(label)

In [35]:
label_count

{0: 56975,
 1: 2009,
 2: 3726,
 3: 1976,
 4: 1394,
 5: 2678,
 6: 3959,
 7: 3579,
 8: 11271,
 9: 6203,
 10: 3278,
 11: 1068,
 12: 1927,
 13: 916,
 14: 1070,
 15: 7268,
 16: 1807}

In [13]:
d1, d2, d3 = input_image.shape

In [15]:
train_data = np.zeros((d1*d2, d3))
index = 0
for i in range(d1):
    for j in range(d2):
        train_data[index, :] = input_image[i, j, :]
        index += 1

In [16]:
train_data

array([[ 290.,  381.,  585., ...,   36.,   11.,   17.],
       [ 290.,  381.,  519., ...,   36.,   11.,   15.],
       [ 290.,  381.,  585., ...,   30.,   11.,   11.],
       ..., 
       [ 447.,  405.,  674., ...,   42.,   11.,   23.],
       [ 447.,  405.,  674., ...,   42.,   17.,   27.],
       [ 376.,  489.,  674., ...,   42.,   15.,   23.]])

In [18]:
train_data = StandardScaler().fit_transform(train_data)

In [19]:
train_data

array([[-1.31588725, -1.23289513, -1.05361719, ..., -0.14195317,
        -0.33707282, -0.38544704],
       [-1.31588725, -1.23289513, -1.69304626, ..., -0.14195317,
        -0.33707282, -0.5310588 ],
       [-1.31588725, -1.23289513, -1.05361719, ..., -0.39967673,
        -0.33707282, -0.82228231],
       ..., 
       [ 1.19797805, -0.93517923, -0.19135678, ...,  0.11577039,
        -0.33707282,  0.05138822],
       [ 1.19797805, -0.93517923, -0.19135678, ...,  0.11577039,
         0.32310714,  0.34261173],
       [ 0.0611345 ,  0.10682641, -0.19135678, ...,  0.11577039,
         0.10304715,  0.05138822]])

In [20]:
train_data.shape

(111104, 204)

In [36]:
not_0 = []
for i in range(len(label)):
    if label[i][0] != 0:
        not_0.append(i)

pca = PCA(20)
pca.fit(train_data)
train_data = pca.transform(train_data)

In [38]:
train_data = train_data[not_0]
label = label[not_0] - 1

In [39]:
train_x, test_x, train_y, test_y = train_test_split(train_data, label, test_size = 0.25, random_state=42)

In [40]:
n_class = 16
train_y = to_categorical(train_y, n_class)

def build_model(n_input):
    inp = Input((n_input, 1))
    x = Reshape((n_input, 1, ))(inp)
    x = Conv1D(filters=128, kernel_size=7, padding='same', activation='relu')(inp)
    # x = MaxPool1D(pool_size=2, strides=2)(x)
    # x = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(x)
    x = Conv1D(filters=16, kernel_size=5, padding='same', activation='relu')(x)
    #x = MaxPool1D(pool_size=2, strides=2)(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    # x = Dropout(0.1)(x)
    x = Dense(16, activation='softmax')(x)
    model = Model(inp, x)
    opt = Adam(lr=0.001)
    model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'],)
    return model

In [45]:
def build_model2(n_input):
    inp = Input((n_input,))
    x = Reshape((1, n_input,))(inp)
    x = Dense(500, activation='relu')(inp)
    x = Dense(350, activation='relu')(x)
    x = Dense(150, activation='relu')(x)
    x = Dense(16, activation='softmax')(x)
    model = Model(inp, x)
    opt = Adam(lr=0.001)
    model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'],)
    return model

In [46]:
model = build_model2(d3)

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 204)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               102500    
_________________________________________________________________
dense_6 (Dense)              (None, 350)               175350    
_________________________________________________________________
dense_7 (Dense)              (None, 150)               52650     
_________________________________________________________________
dense_8 (Dense)              (None, 16)                2416      
Total params: 332,916
Trainable params: 332,916
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(train_x, train_y, epochs=300, batch_size=200, verbose=1)

Epoch 1/300
40596/40596 [==============================] - 1s - loss: 0.4326 - acc: 0.8278     
Epoch 2/300
40596/40596 [==============================] - 1s - loss: 0.2528 - acc: 0.8960     
Epoch 3/300
40596/40596 [==============================] - 1s - loss: 0.2309 - acc: 0.9053     
Epoch 4/300
40596/40596 [==============================] - 1s - loss: 0.1998 - acc: 0.9166     
Epoch 5/300
40596/40596 [==============================] - 1s - loss: 0.1965 - acc: 0.9174     
Epoch 6/300
40596/40596 [==============================] - 1s - loss: 0.1828 - acc: 0.9232     
Epoch 7/300
40596/40596 [==============================] - 1s - loss: 0.1851 - acc: 0.9229     
Epoch 8/300
40596/40596 [==============================] - 1s - loss: 0.1685 - acc: 0.9285     
Epoch 9/300
40596/40596 [==============================] - 1s - loss: 0.1627 - acc: 0.9319     
Epoch 10/300
40596/40596 [==============================] - 1s - loss: 0.1599 - acc: 0.9327     
Epoch 11/300
40596/40596 [=============

40596/40596 [==============================] - 1s - loss: 0.0506 - acc: 0.9802     
Epoch 169/300
40596/40596 [==============================] - 1s - loss: 0.0462 - acc: 0.9822     
Epoch 170/300
40596/40596 [==============================] - 1s - loss: 0.0560 - acc: 0.9783     
Epoch 171/300
40596/40596 [==============================] - 1s - loss: 0.0550 - acc: 0.9788     
Epoch 172/300
40596/40596 [==============================] - 2s - loss: 0.0411 - acc: 0.9837     
Epoch 173/300
40596/40596 [==============================] - 2s - loss: 0.0372 - acc: 0.9855     
Epoch 174/300
40596/40596 [==============================] - 1s - loss: 0.0424 - acc: 0.9837     
Epoch 175/300
40596/40596 [==============================] - 1s - loss: 0.0441 - acc: 0.9821     
Epoch 176/300
40596/40596 [==============================] - 1s - loss: 0.0412 - acc: 0.9834     
Epoch 177/300
40596/40596 [==============================] - 1s - loss: 0.0423 - acc: 0.9837     
Epoch 178/300
40596/40596 [=======

In [49]:
pre = model.predict(test_x)

In [50]:
pre = np.argmax(pre, axis=1)

In [51]:
print(accuracy_score(test_y, pre))

0.961501514816


In [20]:
svm_label = np.argmax(train_y, axis=1)

In [21]:
svm = SVC(gamma = 0.125 ,  C= 16,  random_state=42)

In [22]:
svm.fit(train_x, svm_label)

SVC(C=16, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.125, kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
svm_pre = svm.predict(test_x)

In [24]:
accuracy_score(test_y, svm_pre)

0.60715447154471547